In [1]:
import pandas as pd
from config import get_system_prompt, PERSONA_PROFILE, TEST_SCENARIOS
from evaluator import ModelEvaluator

In [2]:
MODELS = [
    {
        "name": "Gemma-2-2B-It",
        "path": "./models/Gemma-2-2b-it-Q4_K_M.gguf"
    },
    {
        "name": "Llama-3.2-3B-Instruct",
        "path": "./models/Llama-3.2-3B-Instruct-Q4_K_M.gguf"
    },
    {
        "name": "Qwen2.5-3B-Instruct",
        "path": "./models/Qwen2.5-3B-Instruct-Q4_K_M.gguf"
    },
    {
        "name": "EXAONE-3.5-2.4B-Instruct",
        "path": "./models/EXAONE-3.5-2.4B-Instruct-Q4_K_M.gguf"
    }
]

In [3]:
# def main():
#     results = []
#     system_prompt = get_system_prompt(PERSONA_PROFILE)

#     for model in MODELS:
#         evaluator = ModelEvaluator(model['path'], model['name'])
#         evaluator.load_model()
        
#         if not evaluator.llm:
#             continue

#         # 시나리오별 테스트
#         print(f"▶ {model['name']} 테스트")
#         for scenario in TEST_SCENARIOS:
#             result = evaluator.generate_and_measure(system_prompt, scenario)
#             if result:
#                 results.append(result)
#                 print(f"   [Query] {scenario[:30]}...")
#                 print(f"   [Resp]  {result['response'][:30]}...")
#                 print(f"   [Perf]  TTFT: {result['ttft']}s | TPS: {result['tps']}")
        
#         # 모델 언로드 (다음 모델을 위해 메모리 비우기)
#         evaluator.unload_model()

#     # 결과 저장 및 출력
#     df = pd.DataFrame(results)
#     df = df[['model', 'ttft', 'tps', 'input', 'response']]
    
#     print("\n[테스트 결과]")
#     print(df.groupby('model')[['ttft', 'tps']].mean()) # 모델별 평균 성능
    
#     # CSV 저장
#     # df.to_csv("sllm_persona_test_result.csv", index=False, encoding='utf-8-sig')
#     # print("\nResults saved to 'sllm_persona_test_result.csv'")

In [6]:
NUM_ITERATIONS = 5  # 각 질문당 5번 반복
SKIP_COLD = True    # 첫 번째 실행 결과는 통계에서 제외

def main():
    results = []
    system_prompt = get_system_prompt(PERSONA_PROFILE)
    
    for model in MODELS:
        evaluator = ModelEvaluator(model['path'], model['name'])
        evaluator.load_model()

        print(f"▶ {model['name']} 테스트")

        # 시나리오별 반복 테스트
        for scenario_idx, scenario in enumerate(TEST_SCENARIOS):
            for i in range(NUM_ITERATIONS):
                result = evaluator.generate_and_measure(system_prompt, scenario)
                
                if result:
                    # 콜드스타트 제외 로직
                    if SKIP_COLD and i == 0:
                        continue 
                    
                    # 결과에 회차 정보 추가
                    result['iteration'] = i + 1
                    result['scenario_id'] = scenario_idx + 1
                    results.append(result)
                    
                    print(f"      [Iter {i+1}] TTFT: {result['ttft']}s | TPS: {result['tps']}")

        # 모델 언로드
        evaluator.unload_model()

    # 결과 저장 및 분석
    df = pd.DataFrame(results)
    
    print("\n[테스트 결과]")
    # 모델별 전체 평균
    summary = df.groupby('model')[['ttft', 'tps']].agg(['mean', 'std'])
    print(summary)

In [7]:
if __name__ == "__main__":
    main()

모델 로드 중...: Gemma-2-2B-It


llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized
llama_kv_cache_unified_iswa: using full-size SWA cache (ref: https://github.com/ggml-org/llama.cpp/pull/13194#issuecomment-2868343055)


✅ 모델 로드 완료
▶ Gemma-2-2B-It 테스트
      [Iter 2] TTFT: 0.0583s | TPS: 17.7
      [Iter 3] TTFT: 0.0553s | TPS: 17.63
      [Iter 4] TTFT: 0.0563s | TPS: 17.69
      [Iter 5] TTFT: 0.0568s | TPS: 17.62
      [Iter 2] TTFT: 0.0556s | TPS: 17.99
      [Iter 3] TTFT: 0.0561s | TPS: 17.86
      [Iter 4] TTFT: 0.0557s | TPS: 17.67
      [Iter 5] TTFT: 0.0549s | TPS: 17.9
      [Iter 2] TTFT: 0.0561s | TPS: 17.89
      [Iter 3] TTFT: 0.0552s | TPS: 17.94
      [Iter 4] TTFT: 0.0567s | TPS: 17.65
      [Iter 5] TTFT: 0.0607s | TPS: 17.16
🗑️ 모델 해제 완료: Gemma-2-2B-It

모델 로드 중...: Llama-3.2-3B-Instruct


llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


✅ 모델 로드 완료
▶ Llama-3.2-3B-Instruct 테스트
      [Iter 2] TTFT: 0.1285s | TPS: 15.37
      [Iter 3] TTFT: 0.066s | TPS: 15.89
      [Iter 4] TTFT: 0.0613s | TPS: 15.6
      [Iter 5] TTFT: 0.0623s | TPS: 15.84
      [Iter 2] TTFT: 0.063s | TPS: 15.5
      [Iter 3] TTFT: 0.064s | TPS: 15.84
      [Iter 4] TTFT: 0.0639s | TPS: 15.92
      [Iter 5] TTFT: 0.0632s | TPS: 15.75
      [Iter 2] TTFT: 0.0626s | TPS: 15.92
      [Iter 3] TTFT: 0.0625s | TPS: 15.71
      [Iter 4] TTFT: 0.0618s | TPS: 15.69
      [Iter 5] TTFT: 0.0638s | TPS: 15.93
🗑️ 모델 해제 완료: Llama-3.2-3B-Instruct

모델 로드 중...: Qwen2.5-3B-Instruct


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


✅ 모델 로드 완료
▶ Qwen2.5-3B-Instruct 테스트
      [Iter 2] TTFT: 0.0652s | TPS: 14.99
      [Iter 3] TTFT: 0.0612s | TPS: 15.12
      [Iter 4] TTFT: 0.0626s | TPS: 15.08
      [Iter 5] TTFT: 0.0591s | TPS: 16.08
      [Iter 2] TTFT: 0.0608s | TPS: 15.37
      [Iter 3] TTFT: 0.0608s | TPS: 15.68
      [Iter 4] TTFT: 0.0608s | TPS: 15.14
      [Iter 5] TTFT: 0.0617s | TPS: 14.9
      [Iter 2] TTFT: 0.0621s | TPS: 14.75
      [Iter 3] TTFT: 0.0598s | TPS: 15.03
      [Iter 4] TTFT: 0.0647s | TPS: 15.17
      [Iter 5] TTFT: 0.0605s | TPS: 15.63
🗑️ 모델 해제 완료: Qwen2.5-3B-Instruct

모델 로드 중...: EXAONE-3.5-2.4B-Instruct


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


✅ 모델 로드 완료
▶ EXAONE-3.5-2.4B-Instruct 테스트
      [Iter 2] TTFT: 0.0502s | TPS: 20.74
      [Iter 3] TTFT: 0.0486s | TPS: 20.71
      [Iter 4] TTFT: 0.0504s | TPS: 20.8
      [Iter 5] TTFT: 0.05s | TPS: 20.72
      [Iter 2] TTFT: 0.0512s | TPS: 19.26
      [Iter 3] TTFT: 0.0502s | TPS: 20.25
      [Iter 4] TTFT: 0.0493s | TPS: 20.43
      [Iter 5] TTFT: 0.0499s | TPS: 19.42
      [Iter 2] TTFT: 0.0491s | TPS: 20.04
      [Iter 3] TTFT: 0.0524s | TPS: 19.24
      [Iter 4] TTFT: 0.0487s | TPS: 20.24
      [Iter 5] TTFT: 0.0496s | TPS: 20.04
🗑️ 모델 해제 완료: EXAONE-3.5-2.4B-Instruct


[테스트 결과]
                              ttft                  tps          
                              mean       std       mean       std
model                                                            
EXAONE-3.5-2.4B-Instruct  0.049967  0.001068  20.157500  0.578574
Gemma-2-2B-It             0.056475  0.001610  17.725000  0.222036
Llama-3.2-3B-Instruct     0.068575  0.018911  15.746667  0.181024
Qwen2.5-3B-I